### Model_JX

+ attention

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
sys.path.append('/root/Desktop/triton')
from trition import attention  # 导入 attention 函数

class JXdataset(Dataset):
    def __init__(self, path):
        JX = pd.read_csv(path)
        JX_data = torch.tensor(JX.values, dtype=torch.float32)
        self.value, self.target = JX_data[:, :-1], JX_data[:, -1]
    
    def __getitem__(self, index):
        return self.value[index], self.target[index]

    def __len__(self):
        return len(self.value)
    

batch_size = 64

train_dataset = JXdataset("./jixie_train_data.csv")
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = JXdataset("./jixie_test_data.csv")
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_func = nn.MSELoss()

TMA benchmarks will be running with experimental grid constant TMA descriptor.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class MHA(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        # regularization
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        
    def forward(self, x):
        B, T, C = x.size()  # batch size, sequence length (1 for regression), embedding dimensionality
        
        # calculate query, key, values for all heads in batch
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        
        # reshape to (B, nh, T, hs)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # (B, nh, T, hs)
        
        # attention scores
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = F.softmax(att, dim=-1)
        
        # apply attention to values
        y = att @ v
        
        # re-assemble all head outputs
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        
        # output projection
        y = self.c_proj(y)
        return y

class RegressionModel(nn.Module):
    def __init__(self, input_dim=4, hidden_dim=64, n_head=4):
        super().__init__()
        # MHA configuration
        self.config = type('Config', (), {
            'n_embd': hidden_dim,
            'n_head': n_head
        })
        
        # Input embedding
        self.input_embedding = nn.Linear(input_dim, hidden_dim)
        
        # MHA layer
        self.mha = MHA(self.config)
        
        # Output layer
        self.output_layer = nn.Linear(hidden_dim, 1)
        
    def forward(self, x):
        # x: [batch_size, input_features]
        x = x.unsqueeze(1)  # [batch_size, 1, input_features]
        x = self.input_embedding(x)  # [batch_size, 1, hidden_dim]
        x = self.mha(x)  # [batch_size, 1, hidden_dim]
        x = self.output_layer(x)  # [batch_size, 1, 1]
        x = x.squeeze(-1).squeeze(-1)  # [batch_size]
        return x

In [ ]:
# 训练函数
def train_loop(model, train_loader, optimizer, loss_fn, device, epoch):
    model.train()
    total_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1} Training")
    
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()

        # 确保数据形状正确
        data = data.unsqueeze(1)  # 如果需要，确保数据形状为 [batch_size, 1, input_dim]

        output = model(data)  # 前向传播
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pbar.set_postfix({"loss": total_loss / (batch_idx + 1)})

    avg_loss = total_loss / len(train_loader)
    print(f"训练损失: {avg_loss:.6f}")
    return avg_loss


# 测试函数
def test_loop(model, test_loader, loss_fn, device):
    model.eval()
    test_loss = 0.0
    
    with torch.no_grad():
        for data, target in test_loader:
            # 将数据移至设备
            data, target = data.to(device), target.to(device)
            
            # 前向传播
            output = model(data)
            
            # 计算损失
            loss = loss_fn(output, target)
            
            # 更新总损失
            test_loss += loss.item()
    
    avg_loss = test_loss / len(test_loader)
    print(f"测试损失: {avg_loss:.6f}")
    return avg_loss

# 主函数
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 超参数
    input_dim = 4  # 输入特征数
    hidden_dim = 64
    n_head = 4
    learning_rate = 1e-3
    weight_decay = 1e-4
    num_epochs = 100
    
    # 创建模型
    model = RegressionModel(input_dim, hidden_dim, n_head).to(device)
    
    # 定义优化器
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    
    # 训练历史
    train_losses = []
    test_losses = []
    
    # 训练循环
    for epoch in range(num_epochs):
        train_loss = train_loop(model, train_loader, optimizer, loss_func, device, epoch)
        test_loss = test_loop(model, test_loader, loss_func, device)
        
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        
        # 这里可以添加早停逻辑
    
    # 绘制训练和测试损失曲线
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='training loss')
    plt.plot(test_losses, label='test loss')
    plt.xlabel('epoch')
    plt.ylabel('loss (MSE)')
    plt.title('test vs train loss')
    plt.legend()
    plt.grid(True)
    plt.savefig('loss_plot.png')
    
    # 保存模型
    torch.save(model.state_dict(), 'regression_model.pth')
    
    print("训练完成!")

if __name__ == "__main__":
    main()

Epoch 1 Training:   0%|          | 0/7 [00:00<?, ?it/s]

data shape:torch.Size([64, 4])


TypeError: apply() takes no keyword arguments